In [ ]:
!pip install gensim

In [1]:
import re
import numpy as np
import pandas as pd
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.corpus import stopwords


df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

['rec.autos' 'comp.sys.mac.hardware' 'rec.motorcycles' 'misc.forsale'
 'comp.os.ms-windows.misc' 'alt.atheism' 'comp.graphics'
 'rec.sport.baseball' 'rec.sport.hockey' 'sci.electronics' 'sci.space'
 'talk.politics.misc' 'sci.med' 'talk.politics.mideast'
 'soc.religion.christian' 'comp.windows.x' 'comp.sys.ibm.pc.hardware'
 'talk.politics.guns' 'talk.religion.misc' 'sci.crypt']


In [2]:
df.head()

,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
10,From: irwin@cmptrc.lonestar.org (Irwin Arnstei...,8,rec.motorcycles
100,From: tchen@magnus.acs.ohio-state.edu (Tsung-K...,6,misc.forsale
1000,From: dabl2@nlm.nih.gov (Don A.B. Lindbergh)\n...,2,comp.os.ms-windows.misc


In [3]:
data = df.content.values.tolist()

data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
data = [re.sub('\s+', ' ', sent) for sent in data]
data = [re.sub("\'", "", sent) for sent in data]

In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

data_words = list(sent_to_words(data))
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
print(trigram_mod[bigram_mod[data_words[0]]])

/home/kinetik/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


In [5]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [6]:
data_words_nostops = remove_stopwords(data_words)
data_words_bigrams = make_bigrams(data_words_nostops)

In [7]:
id2word = corpora.Dictionary(data_words_bigrams)
texts = data_words_bigrams
corpus = [id2word.doc2bow(text) for text in texts]

In [8]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=1,
                                           alpha='auto',
                                           per_word_topics=True)

In [13]:
predict[0]

([(0, 0.010296087),
  (2, 0.22384112),
  (3, 0.01670613),
  (4, 0.07893116),
  (6, 0.16464594),
  (7, 0.012069532),
  (9, 0.016152646),
  (13, 0.03752394),
  (15, 0.12965761),
  (18, 0.25025794),
  (19, 0.01633805)],
 [(0, [2, 18, 15]),
  (1, [6, 18, 15, 13]),
  (2, [2, 13, 18, 3, 19]),
  (3, [13]),
  (4, [18, 3, 19]),
  (5, [18, 15, 2, 13, 6, 3, 9, 19]),
  (6, [4, 2]),
  (7, [18, 6, 15, 4, 3]),
  (8, [18, 15, 3, 13, 4, 19, 9]),
  (9, [2, 15]),
  (10, [2, 15]),
  (11, [18, 15, 2, 19, 13, 14]),
  (12, [2]),
  (13, [15]),
  (14, [2]),
  (15, [6]),
  (16, [9, 13, 19, 0, 18, 17]),
  (17, [6, 15, 18, 4, 2]),
  (18, [15]),
  (19, [6, 15, 2]),
  (20, [18, 6]),
  (21, [18, 2, 15]),
  (22, [2, 4, 15, 6, 18]),
  (23, [6, 18, 15, 4, 2]),
  (24, [18, 15]),
  (25, [6, 18, 2, 15, 4]),
  (26, [18, 15, 3, 19, 2, 13]),
  (27, [6, 2, 15]),
  (28, [2]),
  (29, [2]),
  (30, [15, 2, 18, 13, 6]),
  (31, [2]),
  (32, [6, 15, 4, 18]),
  (33, [6, 18, 15, 4, 2]),
  (34, [2, 15]),
  (35, [6, 18]),
  (36, [4]),
 

In [12]:
predict = list(doc_lda)

In [9]:
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.025*"year" + 0.011*"game" + 0.010*"games" + 0.008*"last" + 0.008*"teams" + 0.008*"gun" + 0.007*"police" + 0.006*"play" + 0.006*"kids" + 0.005*"points"'), (1, '0.021*"gd" + 0.015*"islam" + 0.012*"moving" + 0.008*"cubs_suck" + 0.007*"mt" + 0.007*"ei" + 0.007*"wt" + 0.007*"ul" + 0.007*"sk" + 0.006*"rc"'), (2, '0.008*"oil" + 0.007*"yo" + 0.007*"disk" + 0.007*"ibm" + 0.007*"reference" + 0.007*"model" + 0.006*"change" + 0.006*"price" + 0.006*"engine" + 0.005*"six"'), (3, '0.009*"president" + 0.008*"went" + 0.007*"told" + 0.007*"second" + 0.006*"day" + 0.006*"home" + 0.005*"th" + 0.005*"mr_stephanopoulos" + 0.005*"us" + 0.005*"house"'), (4, '0.018*"car" + 0.011*"bike" + 0.008*"organization" + 0.008*"cars" + 0.007*"nntp_posting" + 0.007*"writes" + 0.007*"road" + 0.007*"lines" + 0.007*"article" + 0.006*"boston"'), (5, '0.065*"max" + 0.017*"windows" + 0.014*"software" + 0.009*"server" + 0.009*"mb" + 0.009*"using" + 0.009*"files" + 0.009*"version" + 0.008*"program" + 0.008*"system"'), (6,

In [ ]:
import nltk